In [3]:
import cv2 as cv
import numpy as np
# frameWidth = 640
# frameHeight = 480
# cap = cv.VideoCapture(0)
# cap.set(3,frameWidth)
# cap.set(4, frameHeight)
textsize = 2.5
thickness = 10
def empty(a):
    pass

def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range (0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0,0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y],(imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv.cvtColor(imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0,rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0,0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x],(imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
        
    return ver

cv.namedWindow("Parameters")
cv.resizeWindow("Parameters", 640, 240)
cv.createTrackbar("gamma", "Parameters", 23, 100, empty)
cv.createTrackbar("threshold1", "Parameters", 50, 255, empty)
cv.createTrackbar("threshold2", "Parameters", 225, 225, empty)
cv.createTrackbar("image#", "Parameters", 0, 2, empty)

# 0: top left, 1: top right, 2: bottom left, 3: bottom right
def reorder (a):
    b = np.zeros_like(a)
    a = a.reshape((4,2))
    add = a.sum(1)
    b[0] = a[np.argmin(add)]
    b[3] = a[np.argmax(add)]
    diff = np.diff(a, axis = 1)
    b[1] = a[np.argmin(diff)]
    b[2] = a[np.argmax(diff)]
    return b

# tries to set the output image size with max_x - min_x, and max_y - min_y
def size(a):
    a = a.reshape((4,2))
    max = np.argmax(a, axis=0) 
    min = np.argmin(a, axis=0)
    return a[max[0]][0] - a[min[0]][0], a[max[1]][1] - a[min[1]][1]


def getContours(img, imgContour):
    contours, hierarchy = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    imgOut = imgContour.copy()
    shape = [4]
    count = 0

    for cnt in contours:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, 0.08 *peri, True)
        if(area > calculatedArea*.25 and len(approx) == 4):
            count = count +1
            if (count == 1):
                outCon = cnt
            elif (area < cv.contourArea(outCon)):
                outCon = cnt
    if count >0:
        peri = cv.arcLength(outCon, True)
        approx = cv.approxPolyDP(outCon, 0.08 *peri, True)
        area = cv.contourArea(outCon)
        x, y, w, h = cv.boundingRect(approx)

        approxReorder = reorder(approx)
        cv.drawContours(imgContour, outCon, -1, (255, 0,255), thickness)
        cv.putText(imgContour, "Points: " + str(len(approx)), (x + 20, y+70), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "Area: " + str(int(area/calculatedArea*100)) + "%", (x+20, y+170), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "x: " + str(x) + " y: " + str(y), (x+20, y+270), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "width: " + str(x) + "  height: " + str(y), (x+20, y+340), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "p1: " + str(approxReorder[0]), (x-20, y-20), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "p2: " + str(approxReorder[1]), (x+int(w/2), y-20), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "p3: " + str(approxReorder[2]), (x-20, y+h+20), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        cv.putText(imgContour, "p4: " + str(approxReorder[3]), (x+int(w/2), y+h+20), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
        
        for i in range(4):
            imgContour = cv.circle(imgContour, (approxReorder[i][0][0], approxReorder[i][0][1]), 0, (255,0,0), 40)
        pts1 = np.float32(approxReorder)
        approxReorder = approxReorder.reshape((4,2))
        width, height = size(approxReorder)
        approxReorder = approxReorder.reshape((2,4))
        pts2 = np.float32([[0,0], [width,0], [0,height],[width,height]])
        M = cv.getPerspectiveTransform(pts1, pts2)
        dst = cv.warpPerspective(imgOut, M, (width,height))
        dst = cv.resize(dst, (0,0), None, .25, .25)
        cv.imshow("output", dst) 
        cv.putText(imgContour, "count: " + str(count), (0,50), cv.FONT_HERSHEY_COMPLEX, textsize, (0,255,0),thickness)
while True:
    gamma = cv.getTrackbarPos("gamma", "Parameters")
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma/100) * 255.0, 0, 255)
    # success, img = cap.read()
    index = cv.getTrackbarPos("image#", "Parameters")
    img = cv.imread('test1' + str(index) + '.jpg')
    frameHeight, frameWidth, channels = img.shape
    calculatedArea = frameWidth*frameHeight
    imgContour = img.copy()
    imgBlur = cv.GaussianBlur(img, (1,1),1)
    imgGray = cv.cvtColor(imgBlur, cv.COLOR_BGR2GRAY)
    imgGrayGamma = cv.LUT(imgGray, lookUpTable)
    threshold1 = cv.getTrackbarPos("threshold1", "Parameters")
    threshold2 = cv.getTrackbarPos("threshold2", "Parameters")
    imgCanny = cv.Canny(imgGrayGamma, threshold1, threshold2)
    kernel = np.ones((4,4))
    imgDil =  cv.dilate(imgCanny, kernel, iterations = 2)
    kernel = np.ones((3,3))
    imgThreshold = cv.erode(imgDil, kernel, iterations = 2)
    lines = cv.HoughLinesP(imgThreshold,1,np.pi/180,40,minLineLength=30,maxLineGap=30)
    getContours(imgThreshold, imgContour)
    cv.putText(img, "original", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgBlur, "GaussianBlur (7,7)", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgGray, "GrayScale", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgGrayGamma, "Gamma: " + str(gamma/100), (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgCanny, "Canny " + str(threshold1) + " " +str(threshold2), (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgDil, "Dilate (3,3)", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgThreshold, "Threshold (3,3)", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    cv.putText(imgContour, "Contour", (0,100), cv.FONT_HERSHEY_COMPLEX, textsize, (255,0,0),thickness)
    imgStack = stackImages(0.13,([img, imgBlur, imgGray, imgGrayGamma, imgCanny], 
                                [imgDil , imgThreshold, imgThreshold ,imgContour, imgContour]) )
    cv.imshow("result",imgStack)
    imgContour = cv.resize(imgContour, (0,0), None, .25, .25)
    cv.imshow("result1",imgContour)
    if(cv.waitKey(1) & 0xFF == ord('q')):
        cv.destroyAllWindows()
        break